In [1]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw2'))
	print(os.getcwd())
except:
	pass

/home/renjiezhu/school_workspace/recommender_system/hw2


# Homework 2 - Renjie Zhu - A53266114

In [2]:
import scipy as sp
import numpy as np
from sklearn import linear_model
from scipy.io import arff
from io import StringIO

In [3]:
raw, meta = arff.loadarff('5year.arff')
X = raw["class"].astype(int)
A = np.array([(raw["Attr"+str(i+1)]) for i in range(64)]).T
A = np.nan_to_num(A)

In [4]:
# f = open("5year.arff", 'r')

# # Reading in data
# while not '@data' in f.readline():
#     pass

# dataset = []
# for l in f:
#     if '?' in l: # Missing entry
#         continue
#     l = l.split(',')
#     values = [1] + [float(x) for x in l]
#     values[-1] = values[-1] > 0 # Convert to bool
#     dataset.append(values)

# A = np.array([d[:-1] for d in dataset])
# X = np.array([d[-1] for d in dataset])

## 1.

In [5]:
model = linear_model.LogisticRegression(C=1.0, solver='lbfgs',max_iter=10000)
model.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
pred = model.predict(A)
print(f"Accuracy: {sum(pred==X)/len(X)*100}%.")
# sum(pred==X)

Accuracy: 93.29949238578679%.


In [7]:
def BER(pred, X):
    TP = sum(np.logical_and(pred, X))
    FP = sum(np.logical_and(pred, np.logical_not(X)))
    TN = sum(np.logical_and(np.logical_not(pred), np.logical_not(X)))
    FN = sum(np.logical_and(np.logical_not(pred), X))
    return (0.5 * (FN / (FN + TN) + FP / (FP + TP)))

print(f"BER: {BER(pred, X)*100}%.")


BER: 16.632653061224488%.


## 3.

In [8]:
AX = np.concatenate((A,X.reshape(-1,1)),axis=1)
np.random.shuffle(AX)

In [9]:
v = len(AX) // 2
t = len(AX) // 4

test_set = AX[:t]
valid_set = AX[t+1:v]
train_set = AX[v+1:]

A = train_set[:,:-1]
X = train_set[:,-1]


In [10]:
model_balanced = linear_model.LogisticRegression(C=1,class_weight="balanced",solver='lbfgs', max_iter=10000)
model_balanced.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
p_train = model_balanced.predict(A)
p_test = model_balanced.predict(test_set[:,:-1])
p_valid = model_balanced.predict(valid_set[:,:-1])

In [12]:
print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

Training set:	Accuracy: 82.22748815165876%	BER: 39.967373282762274%
Test set:	Accuracy: 83.07379823967501%	BER: 39.99123389639841%
Validation set:	Accuracy: 81.78740690589031%	BER: 40.649676432109146%


## 4.

In [13]:
C = [1E-4, 1E-3, 1E-2, 1E-1, 1, 1E1, 1E2, 1E3, 1E4]
models = [linear_model.LogisticRegression(C=i, class_weight="balanced", solver='lbfgs', max_iter=10000) for i in C]

In [14]:
for c, aModel in zip(C,models):
    aModel.fit(A, X)
    p_train = aModel.predict(A)
    p_test = aModel.predict(test_set[:,:-1])
    p_valid = aModel.predict(valid_set[:,:-1])
    print(f"C = {c}")
    print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
    print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
    print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

C = 0.0001
Training set:	Accuracy: 76.16790792146242%	BER: 42.453917050691246%
Test set:	Accuracy: 75.55856465809072%	BER: 42.51762431601967%
Validation set:	Accuracy: 74.67840216655382%	BER: 43.100239543025616%
C = 0.001
Training set:	Accuracy: 82.80297901150982%	BER: 39.558233804465615%
Test set:	Accuracy: 82.39675016926202%	BER: 40.265004664962945%
Validation set:	Accuracy: 81.24576844955992%	BER: 41.65154744499833%
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C = 0.01
Training set:	Accuracy: 82.46445497630332%	BER: 39.82821417304769%
Test set:	Accuracy: 82.93838862559242%	BER: 40.35033092659447%
Validation set:	Accuracy: 82.19363574813812%	BER: 40.266283366586784%
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of 

Given from the above list of accuracies and BER's, I choose $C=0.1$ for the
classifier with a better performance, because it has lowest validation BER.

## 6. TODO

In [15]:
# weights = [1.0] * len(ytrain)
# mod = linear_model.LogisticRegression(C=1, solver='lbfgs')
# mod.fit(Xtrain, ytrain, sample_weight=weights)

## 7.

In [16]:
from sklearn.decomposition import PCA

In [17]:
pca = PCA(n_components=5)
pca.fit(A)

PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [18]:
pca.components_[0]

array([-6.42980015e-09,  5.55057097e-08,  1.96566429e-08, -2.48804720e-08,
        2.48271408e-03, -7.11758927e-08,  4.25116290e-11, -2.32888951e-07,
       -5.10915373e-08, -6.87357079e-08, -7.43518830e-09, -6.36419906e-08,
       -1.25047834e-08,  4.25116290e-11, -9.99996636e-01, -5.87988750e-08,
       -1.95389197e-07,  4.25116290e-11,  1.33981778e-08, -1.27743302e-04,
       -5.52925793e-07, -6.90317278e-09,  1.41459481e-08, -2.50103693e-08,
       -6.11393361e-08, -5.38264058e-08, -2.74122523e-05, -8.40809015e-07,
        2.89892502e-08,  8.57305863e-08,  1.38371391e-08, -4.90210946e-04,
       -1.19523905e-07, -2.97212700e-07, -6.35285951e-09,  2.68156518e-08,
       -3.34194976e-06, -7.70421181e-08,  5.33765983e-08, -7.62383619e-09,
       -9.37284433e-07,  1.59124263e-08, -3.31630090e-04, -2.03880362e-04,
        1.07267170e-06, -5.59924094e-09, -2.86612245e-05, -3.73186530e-09,
        4.18560196e-08, -6.73838954e-08,  7.63479329e-09, -1.24211268e-06,
       -1.03685559e-06, -

## 8.

In [24]:
for N in range(5,31,5):
    pca = PCA(n_components=N)
    pca.fit(A)
    A_p_train = A @ pca.components_.T
    A_p_valid = valid_set[:,:-1] @ pca.components_.T
    A_p_test = test_set[:,:-1] @ pca.components_.T
    model = linear_model.LogisticRegression(C=1.0, class_weight='balanced', solver='lbfgs')
    model.fit(A_p_train, X)
    print(X)
    p_train = model.predict(A_p_train)
    p_test = model.predict(A_p_test)
    p_valid = model.predict(A_p_valid)
    print(f"N = {N}")
    print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
    print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
    print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

[0. 0. 0. ... 0. 0. 0.]
N = 5
Training set:	Accuracy: 66.65538253215978%	BER: 45.733926273191216%
Test set:	Accuracy: 66.41841570751524%	BER: 45.38248069978461%
Validation set:	Accuracy: 66.82464454976304%	BER: 44.809887296307814%
[0. 0. 0. ... 0. 0. 0.]
N = 10
Training set:	Accuracy: 69.93906567366282%	BER: 44.4607431806227%
Test set:	Accuracy: 69.73595125253892%	BER: 44.053485204902806%
Validation set:	Accuracy: 68.17874069058904%	BER: 45.08208646331517%
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[0. 0. 0. ... 0. 0. 0.]
N = 15
Training set:	Accuracy: 69.36357481381178%	BER: 44.41701706398414%
Test set:	Accuracy: 6